In [5]:
!pip install --upgrade openai

In [6]:
from openai import OpenAI

In [7]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [8]:
import os

In [10]:
# Initialize the OpenAI client with the provided API key
api_key = 'sk-proj-wtF5OrvhNG90rsqMduexT3BlbkFJuqGfql3BxNrT5exppfIe'
client = OpenAI(api_key=api_key)

In [11]:
assistants = client.beta.assistants.list()

print(assistants.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "asst_dMsBJDh1AA6egOwEdDhmDwOg",
      "created_at": 1722288076,
      "description": "This is an assistant created from a notebook.",
      "instructions": "You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
      "metadata": {},
      "model": "gpt-4",
      "name": "Functions-Demo-From-Notebook",
      "object": "assistant",
      "tools": [
        {
          "function": {
            "name": "get_weather",
            "description": "Get the current weather",
            "parameters": {
              "type": "object",
              "properties": {
                "location": {
                  "type": "string",
                  "description": "The city and state e.g. San Francisco, CA"
                },
                "unit": {
                  "type": "string",
                  "enum": [
                    "c",
                    "f"
        

In [13]:
tools = [
    {
        "function": {
            "name": "get_weather",
            "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state e.g. San Francisco, CA"
                        },
                        "unit": {
                            "type": "string",
                            "enum": [
                                "c",
                                "f"
                            ],
                            "description": "The temperature unit to use. Infer this from the user's location."
                        }
                    },
                "required": ["location"]
            },
            "description": "Get the current weather"
        },
        "type": "function"
    },
    {
         "function": {
            "name": "get_n_day_weather_forecast",
            "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state e.g. San Francisco, CA"
                        },
                        "unit": {
                            "type": "string",
                            "enum": [
                                "c",
                                "f"
                            ]
                        },
                        "num_days": {
                            "type": "integer",
                            "description": "The number of days to forecast"
                        }
                    },
                "required": ["location", "num_days"]
            },
            "description": "Get an N-day weather forecast"
        },
        "type": "function"
    }
]

In [14]:
weather_helper = client.beta.assistants.create(
    model='gpt-4',
    name='Functions-Demo-From-Notebook',
    instructions="You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    tools=tools,
    description='This is an assistant created from a notebook.'
)

In [15]:
print(weather_helper.model_dump_json(indent=2))

{
  "id": "asst_3IpML5BNmZcGHdRmFEbBWUmx",
  "created_at": 1722288512,
  "description": "This is an assistant created from a notebook.",
  "instructions": "You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
  "metadata": {},
  "model": "gpt-4",
  "name": "Functions-Demo-From-Notebook",
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "get_weather",
        "description": "Get the current weather",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": [
                "c",
                "f"
              ],
              "description": "The temperature unit to use. Infer this from the user's location."
            

In [16]:
thread = client.beta.threads.create()

In [17]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the weather today in Ahmedabad, Gujarat?"
)

In [18]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=weather_helper.id,
  instructions="Please address the user as Akshat."
)

In [19]:
retrieve_run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [20]:
print(retrieve_run.model_dump_json(indent=2))

{
  "id": "run_bvLzMUgdAvIDX99NSZghWtOR",
  "assistant_id": "asst_3IpML5BNmZcGHdRmFEbBWUmx",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": 1722288565,
  "expires_at": 1722289165,
  "failed_at": null,
  "incomplete_details": null,
  "instructions": "Please address the user as Akshat.",
  "last_error": null,
  "max_completion_tokens": null,
  "max_prompt_tokens": null,
  "metadata": {},
  "model": "gpt-4",
  "object": "thread.run",
  "parallel_tool_calls": true,
  "required_action": {
    "submit_tool_outputs": {
      "tool_calls": [
        {
          "id": "call_p6VPCns0WZOrRm80bru7LAGr",
          "function": {
            "arguments": "{\n  \"location\": \"Ahmedabad, Gujarat\"\n}",
            "name": "get_weather"
          },
          "type": "function"
        }
      ]
    },
    "type": "submit_tool_outputs"
  },
  "response_format": "auto",
  "started_at": 1722288565,
  "status": "requires_action",
  "thread_id": "thread_9MacK3PaisgVY45cT3SYTEPu",
  "tool_c

In [21]:
client.beta.threads.runs.submit_tool_outputs(
    run_id=run.id,
    thread_id=thread.id,
    tool_outputs=[
      {
        "tool_call_id": 'call_p6VPCns0WZOrRm80bru7LAGr',
        "output": "it is 29 degree Celcius",
      }
    ]
)

Run(id='run_bvLzMUgdAvIDX99NSZghWtOR', assistant_id='asst_3IpML5BNmZcGHdRmFEbBWUmx', cancelled_at=None, completed_at=None, created_at=1722288565, expires_at=1722289165, failed_at=None, incomplete_details=None, instructions='Please address the user as Akshat.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1722288565, status='queued', thread_id='thread_9MacK3PaisgVY45cT3SYTEPu', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_weather', description='Get the current weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f'], 'description': "The temperature unit to use. Infer this from the user's location."}}, 'required': ['location']}), type='function'), FunctionTool(function

In [22]:
retrieve_run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [23]:
print(retrieve_run.model_dump_json(indent=2))

{
  "id": "run_bvLzMUgdAvIDX99NSZghWtOR",
  "assistant_id": "asst_3IpML5BNmZcGHdRmFEbBWUmx",
  "cancelled_at": null,
  "completed_at": 1722288643,
  "created_at": 1722288565,
  "expires_at": null,
  "failed_at": null,
  "incomplete_details": null,
  "instructions": "Please address the user as Akshat.",
  "last_error": null,
  "max_completion_tokens": null,
  "max_prompt_tokens": null,
  "metadata": {},
  "model": "gpt-4",
  "object": "thread.run",
  "parallel_tool_calls": true,
  "required_action": null,
  "response_format": "auto",
  "started_at": 1722288641,
  "status": "completed",
  "thread_id": "thread_9MacK3PaisgVY45cT3SYTEPu",
  "tool_choice": "auto",
  "tools": [
    {
      "function": {
        "name": "get_weather",
        "description": "Get the current weather",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state e.g. San Francisco, CA"
    

In [24]:
thread_messages = client.beta.threads.messages.list(thread.id)

In [25]:
user = ''
assistant = ''

for tm in reversed(thread_messages.data):
    content = tm.content[0].text.value
    if tm.role == 'user':
        user += content
    if tm.role == 'assistant':
        assistant += content

In [26]:
print(user)

What is the weather today in Ahmedabad, Gujarat?


In [27]:
print(assistant)

Today's weather in Ahmedabad, Gujarat is 29 degrees Celsius, Akshat.
